# 4.2.1 模板匹配 (Template Matching)

**WBS 4.2.1**: 模板匹配基礎與應用

本模組涵蓋:
- **模板匹配基礎原理**: 滑動窗口與相似度計算
- **六種匹配方法**: TM_CCOEFF, TM_CCORR, TM_SQDIFF 及其歸一化版本
- **多尺度模板匹配**: 解決尺度變化問題
- **多目標檢測**: 使用非極大值抑制 (NMS) 檢測多個目標
- **實戰應用**: Logo檢測、多目標檢測、不同光照條件匹配
- **限制與解決方案**: 旋轉、尺度、光照變化的應對策略

<div style="page-break-after: always"></div>

# 1. 模板匹配基礎原理

## 1-1: 什麼是模板匹配?

> 模板匹配 (Template Matching) 是一種在較大圖像中搜尋和定位模板圖像位置的方法。
>
> **核心思想**: 將模板圖像在原圖上滑動，計算每個位置的相似度，找出最相似的位置。

## 1-2: 工作原理

> **滑動窗口機制**:
> 1. 從原圖左上角開始，將模板圖像在原圖上滑動
> 2. 在每個位置計算模板與原圖對應區域的相似度
> 3. 將相似度結果存儲在結果矩陣中
> 4. 找出結果矩陣中的最大值或最小值位置（取決於匹配方法）
>
> **結果矩陣尺寸**:
> - 原圖尺寸: W × H
> - 模板尺寸: w × h  
> - 結果矩陣: (W-w+1) × (H-h+1)

## 1-3: 應用場景

> **適用場景**:
> * ✅ Logo 檢測與定位
> * ✅ 簡單物體檢測
> * ✅ UI 自動化測試（圖標定位）
> * ✅ 遊戲開發（地圖匹配）
> * ✅ 工業檢測（缺陷檢測）
>
> **不適用場景**:
> * ❌ 物體有旋轉變化
> * ❌ 物體有尺度變化（需要多尺度匹配）
> * ❌ 複雜背景下的物體識別
> * ❌ 需要高度魯棒性的應用

## 1-4: 優缺點分析

> | 特性 | 優點 | 缺點 |
> |------|------|------|
> | **實現難度** | ✅ 簡單易用，OpenCV 內建 | - |
> | **計算速度** | ✅ 相對快速 | ❌ 大圖像計算量大 |
> | **旋轉不變性** | - | ❌ 無旋轉不變性 |
> | **尺度不變性** | - | ❌ 無尺度不變性 |
> | **光照變化** | 部分方法魯棒 | ❌ 需選擇適當方法 |
> | **精確度** | ✅ 固定模板檢測精確 | ❌ 環境變化敏感 |

In [ ]:
# Import necessary libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import time

# Configure matplotlib for better display
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 10
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("Libraries imported successfully")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

<div style="page-break-after: always"></div>

# 2. OpenCV 模板匹配方法

## 2-1: cv2.matchTemplate() 函數

```python
result = cv2.matchTemplate(image, template, method)
```

> **參數說明**:
> * `image`: 原始圖像（被搜尋的圖像）
> * `template`: 模板圖像（要查找的圖像）
> * `method`: 匹配方法（共6種）
>
> **返回值**:
> * `result`: 匹配結果矩陣，每個元素表示對應位置的相似度
> * 結果尺寸: (W-w+1) × (H-h+1)

## 2-2: 六種匹配方法詳解

### 方法 1: TM_SQDIFF (平方差匹配)

> **公式**:
> $$R(x,y) = \sum_{x',y'}[T(x',y') - I(x+x',y+y')]^2$$
>
> **特點**:
> * 計算模板與圖像對應區域的平方差
> * **值越小表示匹配越好**（0 表示完美匹配）
> * 對亮度變化敏感

### 方法 2: TM_SQDIFF_NORMED (歸一化平方差匹配)

> **公式**:
> $$R(x,y) = \frac{\sum_{x',y'}[T(x',y') - I(x+x',y+y')]^2}{\sqrt{\sum_{x',y'}T(x',y')^2 \cdot \sum_{x',y'}I(x+x',y+y')^2}}$$
>
> **特點**:
> * 歸一化版本，結果範圍 [0, 1]
> * **值越小越好**
> * 對亮度變化較不敏感

### 方法 3: TM_CCORR (相關匹配)

> **公式**:
> $$R(x,y) = \sum_{x',y'}[T(x',y') \cdot I(x+x',y+y')]$$
>
> **特點**:
> * 計算模板與圖像的互相關
> * **值越大表示匹配越好**
> * 對亮度變化非常敏感

### 方法 4: TM_CCORR_NORMED (歸一化相關匹配)

> **公式**:
> $$R(x,y) = \frac{\sum_{x',y'}[T(x',y') \cdot I(x+x',y+y')]}{\sqrt{\sum_{x',y'}T(x',y')^2 \cdot \sum_{x',y'}I(x+x',y+y')^2}}$$
>
> **特點**:
> * 歸一化版本，結果範圍 [-1, 1]
> * **值越大越好**
> * 對亮度變化有一定抵抗力

### 方法 5: TM_CCOEFF (相關係數匹配) ⭐

> **公式**:
> $$R(x,y) = \sum_{x',y'}[T'(x',y') \cdot I'(x+x',y+y')]$$
>
> 其中 $T'$ 和 $I'$ 是去除均值後的圖像:
> $$T'(x',y') = T(x',y') - \frac{1}{wh}\sum_{x'',y''}T(x'',y'')$$
>
> **特點**:
> * 去除均值，考慮圖像的相對變化
> * **值越大越好**
> * **對亮度變化魯棒** ✅

### 方法 6: TM_CCOEFF_NORMED (歸一化相關係數匹配) ⭐⭐⭐

> **公式**:
> $$R(x,y) = \frac{\sum_{x',y'}[T'(x',y') \cdot I'(x+x',y+y')]}{\sqrt{\sum_{x',y'}T'(x',y')^2 \cdot \sum_{x',y'}I'(x+x',y+y')^2}}$$
>
> **特點**:
> * 歸一化版本，結果範圍 [-1, 1]
> * **值越大越好**（1 表示完美匹配）
> * **最常用、最魯棒的方法** ✅✅✅
> * 對光照、亮度變化不敏感

## 2-3: 六種方法比較表

> | 方法 | 結果範圍 | 最佳值 | 歸一化 | 對光照敏感度 | 推薦度 |
> |------|---------|--------|--------|-------------|--------|
> | **TM_SQDIFF** | [0, ∞) | 最小值 | ❌ | 高 | ⭐ |
> | **TM_SQDIFF_NORMED** | [0, 1] | 最小值 | ✅ | 中 | ⭐⭐ |
> | **TM_CCORR** | [0, ∞) | 最大值 | ❌ | 非常高 | ⭐ |
> | **TM_CCORR_NORMED** | [0, 1] | 最大值 | ✅ | 高 | ⭐⭐ |
> | **TM_CCOEFF** | (-∞, ∞) | 最大值 | ❌ | 低 | ⭐⭐⭐ |
> | **TM_CCOEFF_NORMED** | [-1, 1] | 最大值 | ✅ | **很低** | ⭐⭐⭐⭐⭐ |

## 2-4: 基礎模板匹配實作

In [ ]:
# Load test image and template
# We'll create a simple test case first
img = cv2.imread('../assets/images/basic/lenaColor.png')

if img is None:
    # Create a test image with colored squares
    img = np.ones((400, 600, 3), dtype=np.uint8) * 200
    cv2.rectangle(img, (50, 50), (200, 200), (255, 0, 0), -1)  # Blue square
    cv2.rectangle(img, (250, 150), (400, 300), (0, 255, 0), -1)  # Green square
    cv2.rectangle(img, (450, 250), (550, 350), (0, 0, 255), -1)  # Red square
    print("Created test image")
else:
    print(f"Loaded image: {img.shape}")

# Extract a template from the image (top-left region)
template = img[50:150, 50:150].copy()

# Get dimensions
h, w = template.shape[:2]
print(f"Template size: {w}x{h}")

# Apply template matching using TM_CCOEFF_NORMED (recommended method)
result = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)

# Find the location with highest correlation
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

# For TM_CCOEFF_NORMED, we need the maximum value
top_left = max_loc
bottom_right = (top_left[0] + w, top_left[1] + h)

# Draw rectangle on the result
img_result = img.copy()
cv2.rectangle(img_result, top_left, bottom_right, (0, 255, 0), 3)

# Display results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(template, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Template', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(result, cmap='hot')
axes[1, 0].set_title('Matching Result Heatmap', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title(f'Detected Location (Score: {max_val:.3f})', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"\nMatching Score: {max_val:.4f}")
print(f"Best Match Location: {top_left}")
print(f"Result matrix shape: {result.shape}")

<div style="page-break-after: always"></div>

# 3. 六種匹配方法視覺對比

## 3-1: 所有方法的匹配結果比較

In [ ]:
# Compare all six matching methods
methods = [
    ('TM_CCOEFF_NORMED', cv2.TM_CCOEFF_NORMED, 'max'),
    ('TM_CCOEFF', cv2.TM_CCOEFF, 'max'),
    ('TM_CCORR_NORMED', cv2.TM_CCORR_NORMED, 'max'),
    ('TM_CCORR', cv2.TM_CCORR, 'max'),
    ('TM_SQDIFF_NORMED', cv2.TM_SQDIFF_NORMED, 'min'),
    ('TM_SQDIFF', cv2.TM_SQDIFF, 'min')
]

fig, axes = plt.subplots(3, 2, figsize=(14, 16))
axes = axes.ravel()

results_summary = []

for idx, (method_name, method, extremum) in enumerate(methods):
    # Apply template matching
    result = cv2.matchTemplate(img, template, method)
    
    # Find best match location
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    
    # Determine which value to use based on method
    if extremum == 'min':
        match_val = min_val
        top_left = min_loc
    else:
        match_val = max_val
        top_left = max_loc
    
    bottom_right = (top_left[0] + w, top_left[1] + h)
    
    # Draw rectangle
    img_display = img.copy()
    cv2.rectangle(img_display, top_left, bottom_right, (0, 255, 0), 3)
    
    # Add method name and score to image
    text = f"{method_name}: {match_val:.3f}"
    cv2.putText(img_display, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 255, 0), 2)
    
    # Display
    axes[idx].imshow(cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB))
    axes[idx].set_title(f'{method_name}\nScore: {match_val:.4f}', 
                       fontsize=11, fontweight='bold')
    axes[idx].axis('off')
    
    # Store results
    results_summary.append({
        'method': method_name,
        'score': match_val,
        'location': top_left
    })

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*70)
print("六種匹配方法結果總結")
print("="*70)
print(f"{'方法':<25} {'匹配分數':<15} {'位置 (x, y)'}")
print("-"*70)
for result in results_summary:
    print(f"{result['method']:<25} {result['score']:>10.4f}     {result['location']}")
print("="*70)

## 3-2: 匹配熱圖視覺化

In [ ]:
# Visualize matching heatmaps for all methods
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, (method_name, method, extremum) in enumerate(methods):
    # Apply template matching
    result = cv2.matchTemplate(img, template, method)
    
    # Normalize result for visualization
    result_normalized = cv2.normalize(result, None, 0, 255, cv2.NORM_MINMAX)
    
    # Display heatmap
    im = axes[idx].imshow(result, cmap='hot', interpolation='nearest')
    axes[idx].set_title(f'{method_name}\n({extremum.upper()} is best)', 
                       fontsize=10, fontweight='bold')
    axes[idx].axis('off')
    
    # Add colorbar
    plt.colorbar(im, ax=axes[idx], fraction=0.046, pad=0.04)

plt.suptitle('匹配方法熱圖比較 (亮處表示高相似度)', 
             fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("\n💡 觀察要點:")
print("- TM_CCOEFF_NORMED: 熱圖最清晰，峰值最明顯")
print("- TM_SQDIFF 系列: 值越小越好（熱圖中暗處為最佳匹配）")
print("- 歸一化方法: 結果範圍統一，更易於設定閾值")

<div style="page-break-after: always"></div>

# 4. 光照變化魯棒性測試

## 4-1: 不同光照條件下的匹配表現

In [ ]:
# Test robustness to illumination changes
# Create variations of the image with different brightness levels
brightness_factors = [0.5, 0.7, 1.0, 1.3, 1.5]  # 50%, 70%, 100%, 130%, 150%

# Test with three representative methods
test_methods = [
    ('TM_CCOEFF_NORMED', cv2.TM_CCOEFF_NORMED, 'max'),
    ('TM_CCORR_NORMED', cv2.TM_CCORR_NORMED, 'max'),
    ('TM_SQDIFF_NORMED', cv2.TM_SQDIFF_NORMED, 'min')
]

# Store results
illumination_results = {method[0]: [] for method in test_methods}

fig, axes = plt.subplots(len(test_methods), len(brightness_factors), 
                        figsize=(16, 10))

for method_idx, (method_name, method, extremum) in enumerate(test_methods):
    for bright_idx, factor in enumerate(brightness_factors):
        # Create brightness-adjusted image
        img_adjusted = np.clip(img * factor, 0, 255).astype(np.uint8)
        
        # Apply template matching
        result = cv2.matchTemplate(img_adjusted, template, method)
        
        # Find best match
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        if extremum == 'min':
            match_val = min_val
            top_left = min_loc
        else:
            match_val = max_val
            top_left = max_loc
        
        bottom_right = (top_left[0] + w, top_left[1] + h)
        
        # Draw result
        img_display = img_adjusted.copy()
        cv2.rectangle(img_display, top_left, bottom_right, (0, 255, 0), 2)
        
        # Display
        axes[method_idx, bright_idx].imshow(cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB))
        axes[method_idx, bright_idx].set_title(
            f'{int(factor*100)}% Brightness\nScore: {match_val:.3f}',
            fontsize=9
        )
        axes[method_idx, bright_idx].axis('off')
        
        # Store score
        illumination_results[method_name].append(match_val)
    
    # Add method name to the left
    axes[method_idx, 0].set_ylabel(method_name, fontsize=11, fontweight='bold')

plt.suptitle('光照變化魯棒性測試', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Plot score variations
fig, ax = plt.subplots(figsize=(10, 6))

for method_name, scores in illumination_results.items():
    ax.plot(brightness_factors, scores, marker='o', linewidth=2, 
           markersize=8, label=method_name)

ax.set_xlabel('亮度係數', fontsize=12, fontweight='bold')
ax.set_ylabel('匹配分數', fontsize=12, fontweight='bold')
ax.set_title('不同匹配方法對光照變化的魯棒性', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, linestyle='--')
ax.axvline(x=1.0, color='red', linestyle='--', alpha=0.5, label='原始亮度')

plt.tight_layout()
plt.show()

print("\n💡 結論:")
print("- TM_CCOEFF_NORMED: 對光照變化最魯棒，分數波動最小 ✅")
print("- TM_CCORR_NORMED: 對光照變化較敏感，分數波動較大")
print("- TM_SQDIFF_NORMED: 中等魯棒性")
print("\n推薦: 實際應用中優先使用 TM_CCOEFF_NORMED")

<div style="page-break-after: always"></div>

# 5. 多目標檢測

## 5-1: 檢測多個相同目標

> **問題**: `cv2.matchTemplate()` 只能找到單一最佳匹配位置
>
> **解決方案**: 
> 1. 設定閾值，找出所有超過閾值的位置
> 2. 使用非極大值抑制 (NMS) 去除重複檢測

## 5-2: 非極大值抑制 (Non-Maximum Suppression)

In [ ]:
# Create an image with multiple instances of the same object
img_multi = np.ones((500, 700, 3), dtype=np.uint8) * 220

# Create a simple template (a colored circle)
template_multi = np.ones((60, 60, 3), dtype=np.uint8) * 220
cv2.circle(template_multi, (30, 30), 25, (0, 0, 255), -1)
cv2.circle(template_multi, (30, 30), 15, (255, 255, 0), -1)

# Place template at multiple locations
locations = [(100, 100), (300, 150), (500, 200), (150, 350), (400, 380)]
for loc in locations:
    x, y = loc
    img_multi[y:y+60, x:x+60] = template_multi

print(f"Created test image with {len(locations)} target instances")
print(f"Image size: {img_multi.shape}")
print(f"Template size: {template_multi.shape}")

In [ ]:
# Apply template matching
gray_img = cv2.cvtColor(img_multi, cv2.COLOR_BGR2GRAY)
gray_template = cv2.cvtColor(template_multi, cv2.COLOR_BGR2GRAY)

result = cv2.matchTemplate(gray_img, gray_template, cv2.TM_CCOEFF_NORMED)
h_t, w_t = gray_template.shape

# Set threshold
threshold = 0.8

# Find all locations above threshold
locations_found = np.where(result >= threshold)
locations_found = list(zip(*locations_found[::-1]))  # (x, y) format

print(f"\nFound {len(locations_found)} locations above threshold {threshold}")

# Draw all detections (before NMS)
img_before_nms = img_multi.copy()
for pt in locations_found:
    cv2.rectangle(img_before_nms, pt, (pt[0] + w_t, pt[1] + h_t), 
                 (0, 255, 0), 2)

# Display before NMS
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(cv2.cvtColor(img_multi, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(result, cmap='hot')
axes[1].set_title('Matching Heatmap', fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_before_nms, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'All Detections (Before NMS)\nTotal: {len(locations_found)}', 
                 fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\n⚠️ 問題: 檢測到許多重疊的矩形框")
print("解決方案: 使用非極大值抑制 (NMS) 去除重複檢測")

In [ ]:
def non_max_suppression(boxes, scores, threshold=0.3):
    """
    Apply Non-Maximum Suppression to remove overlapping boxes
    
    Parameters:
    -----------
    boxes : list of tuples
        List of bounding boxes [(x1, y1, x2, y2), ...]
    scores : list of floats
        Confidence scores for each box
    threshold : float
        IoU threshold for suppression
    
    Returns:
    --------
    keep : list
        Indices of boxes to keep
    """
    if len(boxes) == 0:
        return []
    
    boxes = np.array(boxes).astype(float)
    scores = np.array(scores)
    
    # Get coordinates
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    
    # Calculate area
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    
    # Sort by scores
    order = scores.argsort()[::-1]
    
    keep = []
    
    while order.size > 0:
        i = order[0]
        keep.append(i)
        
        # Calculate IoU with remaining boxes
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        
        intersection = w * h
        iou = intersection / (areas[i] + areas[order[1:]] - intersection)
        
        # Keep boxes with IoU less than threshold
        inds = np.where(iou <= threshold)[0]
        order = order[inds + 1]
    
    return keep

# Prepare boxes and scores for NMS
boxes = []
scores = []

for pt in locations_found:
    x1, y1 = pt
    x2, y2 = x1 + w_t, y1 + h_t
    boxes.append([x1, y1, x2, y2])
    scores.append(result[y1, x1])

# Apply NMS
keep_indices = non_max_suppression(boxes, scores, threshold=0.3)

print(f"\nAfter NMS: {len(keep_indices)} detections remain")
print(f"Removed: {len(boxes) - len(keep_indices)} overlapping detections")

# Draw final detections
img_after_nms = img_multi.copy()
for idx in keep_indices:
    x1, y1, x2, y2 = boxes[idx]
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    cv2.rectangle(img_after_nms, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # Add confidence score
    cv2.putText(img_after_nms, f'{scores[idx]:.2f}', (x1, y1-5),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(cv2.cvtColor(img_before_nms, cv2.COLOR_BGR2RGB))
axes[0].set_title(f'Before NMS\nDetections: {len(locations_found)}', 
                 fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_after_nms, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'After NMS\nDetections: {len(keep_indices)}', 
                 fontsize=12, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ NMS 成功去除重複檢測!")
print(f"原始檢測數: {len(locations_found)}")
print(f"NMS 後檢測數: {len(keep_indices)}")
print(f"去除比例: {(1 - len(keep_indices)/len(locations_found))*100:.1f}%")

<div style="page-break-after: always"></div>

# 6. 多尺度模板匹配

## 6-1: 尺度不變性問題

> **問題**: 標準模板匹配無法處理尺度變化
>
> **解決方案**: 
> - 在多個尺度上進行模板匹配
> - 記錄每個尺度的最佳匹配
> - 選擇所有尺度中的最佳結果

## 6-2: 多尺度匹配實作

In [ ]:
def multi_scale_template_matching(image, template, scales, method=cv2.TM_CCOEFF_NORMED):
    """
    Perform template matching across multiple scales
    
    Parameters:
    -----------
    image : numpy.ndarray
        Input image
    template : numpy.ndarray
        Template image
    scales : list
        List of scale factors to try
    method : int
        OpenCV matching method
    
    Returns:
    --------
    best_match : dict
        Dictionary containing best match information
    all_results : list
        List of all scale results
    """
    best_match = {
        'score': -np.inf,
        'location': None,
        'scale': None,
        'size': None
    }
    
    all_results = []
    
    # Get original template dimensions
    tH, tW = template.shape[:2]
    
    for scale in scales:
        # Resize template
        resized_template = cv2.resize(template, None, fx=scale, fy=scale)
        rH, rW = resized_template.shape[:2]
        
        # Skip if resized template is larger than image
        if rW > image.shape[1] or rH > image.shape[0]:
            continue
        
        # Apply template matching
        result = cv2.matchTemplate(image, resized_template, method)
        
        # Find best match for this scale
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # For TM_CCOEFF_NORMED, higher is better
        score = max_val
        location = max_loc
        
        # Store result
        scale_result = {
            'scale': scale,
            'score': score,
            'location': location,
            'size': (rW, rH)
        }
        all_results.append(scale_result)
        
        # Update best match
        if score > best_match['score']:
            best_match = scale_result.copy()
    
    return best_match, all_results

# Create test image with scaled template
img_scaled = np.ones((600, 800, 3), dtype=np.uint8) * 200

# Create a distinctive template
template_scale_test = np.ones((80, 80, 3), dtype=np.uint8) * 200
cv2.rectangle(template_scale_test, (10, 10), (70, 70), (255, 0, 0), -1)
cv2.rectangle(template_scale_test, (25, 25), (55, 55), (0, 255, 0), -1)
cv2.circle(template_scale_test, (40, 40), 10, (0, 0, 255), -1)

# Place scaled versions in the image
scale_factors = [0.5, 0.75, 1.0, 1.25, 1.5]
positions = [(100, 100), (250, 200), (400, 150), (550, 300), (150, 400)]

for scale, pos in zip(scale_factors, positions):
    scaled_tmpl = cv2.resize(template_scale_test, None, fx=scale, fy=scale)
    h_s, w_s = scaled_tmpl.shape[:2]
    x, y = pos
    
    # Ensure it fits in the image
    if x + w_s <= img_scaled.shape[1] and y + h_s <= img_scaled.shape[0]:
        img_scaled[y:y+h_s, x:x+w_s] = scaled_tmpl

print("Created test image with multiple scaled instances")
print(f"Template size: {template_scale_test.shape[:2]}")
print(f"Scales used: {scale_factors}")

In [ ]:
# Convert to grayscale for matching
gray_img_scaled = cv2.cvtColor(img_scaled, cv2.COLOR_BGR2GRAY)
gray_template_scaled = cv2.cvtColor(template_scale_test, cv2.COLOR_BGR2GRAY)

# Define scales to search
scales_to_search = np.linspace(0.3, 2.0, 20)

# Perform multi-scale matching
print("\nPerforming multi-scale template matching...")
start_time = time.time()

best_match, all_results = multi_scale_template_matching(
    gray_img_scaled, 
    gray_template_scaled, 
    scales_to_search
)

elapsed = time.time() - start_time
print(f"Completed in {elapsed:.2f} seconds")

# Display results
print(f"\nBest Match:")
print(f"  Scale: {best_match['scale']:.2f}")
print(f"  Score: {best_match['score']:.4f}")
print(f"  Location: {best_match['location']}")
print(f"  Size: {best_match['size']}")

# Draw best match
img_best_match = img_scaled.copy()
x, y = best_match['location']
w, h = best_match['size']
cv2.rectangle(img_best_match, (x, y), (x+w, y+h), (0, 255, 0), 3)
cv2.putText(img_best_match, f"Scale: {best_match['scale']:.2f}", (x, y-10),
           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# Draw all detected instances
img_all_detections = img_scaled.copy()
threshold_multi = 0.7

for result in all_results:
    if result['score'] >= threshold_multi:
        x, y = result['location']
        w, h = result['size']
        cv2.rectangle(img_all_detections, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Display
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].imshow(cv2.cvtColor(img_scaled, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image\n(Multiple Scales)', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_best_match, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Best Match\nScale: {best_match["scale"]:.2f}, Score: {best_match["score"]:.3f}', 
                 fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_all_detections, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'All Detections (Score > {threshold_multi})', 
                 fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Plot matching scores across scales
scales_list = [r['scale'] for r in all_results]
scores_list = [r['score'] for r in all_results]

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(scales_list, scores_list, marker='o', linewidth=2, markersize=6)
ax.axhline(y=threshold_multi, color='r', linestyle='--', label=f'Threshold ({threshold_multi})')
ax.axvline(x=best_match['scale'], color='g', linestyle='--', 
          label=f'Best Scale ({best_match["scale"]:.2f})')

ax.set_xlabel('Scale Factor', fontsize=12, fontweight='bold')
ax.set_ylabel('Matching Score', fontsize=12, fontweight='bold')
ax.set_title('Matching Score vs Scale Factor', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print("\n💡 觀察:")
print("- 匹配分數在正確尺度附近達到峰值")
print("- 多尺度匹配可以找到不同大小的目標")
print("- 計算成本隨搜尋尺度數量增加而增加")

<div style="page-break-after: always"></div>

# 7. 實戰應用案例

## 7-1: 應用1 - Logo 檢測

In [ ]:
# Create a sample scene with multiple logos
scene = np.ones((500, 700, 3), dtype=np.uint8) * 240

# Create a simple logo template
logo_template = np.ones((50, 50, 3), dtype=np.uint8) * 240
cv2.circle(logo_template, (25, 25), 20, (0, 0, 200), -1)
cv2.circle(logo_template, (25, 25), 12, (240, 240, 240), -1)
cv2.putText(logo_template, 'CV', (13, 32), cv2.FONT_HERSHEY_SIMPLEX, 
           0.6, (0, 0, 200), 2)

# Place logos at various locations with slight variations
logo_positions = [(50, 50), (200, 100), (400, 150), (100, 300), (500, 350), (300, 400)]

for pos in logo_positions:
    x, y = pos
    # Add slight brightness variation
    brightness = np.random.uniform(0.9, 1.1)
    logo_variant = np.clip(logo_template * brightness, 0, 255).astype(np.uint8)
    
    h_l, w_l = logo_variant.shape[:2]
    if x + w_l <= scene.shape[1] and y + h_l <= scene.shape[0]:
        scene[y:y+h_l, x:x+w_l] = logo_variant

# Add some noise and other elements
cv2.rectangle(scene, (550, 50), (650, 150), (100, 200, 100), 2)
cv2.circle(scene, (600, 300), 30, (200, 100, 100), 3)

print("Created scene with multiple logos")
print(f"Number of logos: {len(logo_positions)}")

In [ ]:
# Detect all logos
gray_scene = cv2.cvtColor(scene, cv2.COLOR_BGR2GRAY)
gray_logo = cv2.cvtColor(logo_template, cv2.COLOR_BGR2GRAY)

# Apply template matching
result_logo = cv2.matchTemplate(gray_scene, gray_logo, cv2.TM_CCOEFF_NORMED)

# Find all matches above threshold
threshold_logo = 0.75
locations_logo = np.where(result_logo >= threshold_logo)
locations_logo = list(zip(*locations_logo[::-1]))

h_logo, w_logo = gray_logo.shape

# Prepare for NMS
boxes_logo = []
scores_logo = []

for pt in locations_logo:
    x1, y1 = pt
    x2, y2 = x1 + w_logo, y1 + h_logo
    boxes_logo.append([x1, y1, x2, y2])
    scores_logo.append(result_logo[y1, x1])

# Apply NMS
keep_logo = non_max_suppression(boxes_logo, scores_logo, threshold=0.3)

# Draw detections
scene_detected = scene.copy()
for idx in keep_logo:
    x1, y1, x2, y2 = boxes_logo[idx]
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    cv2.rectangle(scene_detected, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(scene_detected, f'{scores_logo[idx]:.2f}', (x1, y1-5),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].imshow(cv2.cvtColor(logo_template, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Logo Template', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(scene, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Scene with Multiple Logos', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(result_logo, cmap='hot')
axes[1, 0].set_title('Matching Heatmap', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(scene_detected, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title(f'Detected Logos: {len(keep_logo)}', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"\n✅ Logo 檢測完成!")
print(f"檢測到的 Logo 數量: {len(keep_logo)}")
print(f"實際 Logo 數量: {len(logo_positions)}")
print(f"檢測準確率: {len(keep_logo)/len(logo_positions)*100:.1f}%")

## 7-2: 應用2 - UI 元素定位 (自動化測試)

In [ ]:
# Create a simple UI mockup
ui_screen = np.ones((600, 800, 3), dtype=np.uint8) * 250

# Add UI elements
# Title bar
cv2.rectangle(ui_screen, (0, 0), (800, 60), (200, 200, 200), -1)
cv2.putText(ui_screen, 'Application Title', (20, 40), 
           cv2.FONT_HERSHEY_SIMPLEX, 1.2, (50, 50, 50), 2)

# Buttons
button_positions = [(100, 150), (300, 150), (500, 150),
                   (100, 250), (300, 250), (500, 250)]
button_labels = ['Save', 'Load', 'Export', 'Delete', 'Cancel', 'OK']

for pos, label in zip(button_positions, button_labels):
    x, y = pos
    cv2.rectangle(ui_screen, (x, y), (x+150, y+60), (100, 150, 200), -1)
    cv2.rectangle(ui_screen, (x, y), (x+150, y+60), (70, 120, 170), 3)
    
    # Center text
    text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
    text_x = x + (150 - text_size[0]) // 2
    text_y = y + (60 + text_size[1]) // 2
    cv2.putText(ui_screen, label, (text_x, text_y),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

# Create button template (OK button)
ok_button_template = ui_screen[250:310, 500:650].copy()

print("Created UI mockup")
print(f"UI size: {ui_screen.shape}")
print(f"Button template size: {ok_button_template.shape}")

In [ ]:
# Locate the OK button
gray_ui = cv2.cvtColor(ui_screen, cv2.COLOR_BGR2GRAY)
gray_button = cv2.cvtColor(ok_button_template, cv2.COLOR_BGR2GRAY)

result_ui = cv2.matchTemplate(gray_ui, gray_button, cv2.TM_CCOEFF_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result_ui)

h_btn, w_btn = gray_button.shape
top_left_btn = max_loc
bottom_right_btn = (top_left_btn[0] + w_btn, top_left_btn[1] + h_btn)

# Calculate button center (for clicking)
button_center = (
    top_left_btn[0] + w_btn // 2,
    top_left_btn[1] + h_btn // 2
)

# Visualize detection
ui_detected = ui_screen.copy()
cv2.rectangle(ui_detected, top_left_btn, bottom_right_btn, (0, 255, 0), 3)
cv2.circle(ui_detected, button_center, 5, (255, 0, 0), -1)
cv2.putText(ui_detected, 'Click Here', (button_center[0]-50, button_center[1]-20),
           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

# Display
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].imshow(cv2.cvtColor(ui_screen, cv2.COLOR_BGR2RGB))
axes[0].set_title('UI Screen', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(ok_button_template, cv2.COLOR_BGR2RGB))
axes[1].set_title('Target Button Template', fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(ui_detected, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'Button Located (Score: {max_val:.3f})', fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"\n✅ UI 元素定位成功!")
print(f"按鈕位置: {top_left_btn}")
print(f"點擊座標: {button_center}")
print(f"匹配分數: {max_val:.4f}")
print("\n應用場景: 自動化測試、RPA、遊戲機器人等")

<div style="page-break-after: always"></div>

# 8. 性能基準測試

## 8-1: 不同匹配方法的性能比較

In [ ]:
# Performance benchmarking
num_iterations = 50

print(f"Performance Benchmark ({num_iterations} iterations)")
print("="*70)

timing_results = []

for method_name, method, _ in methods:
    start = time.time()
    
    for _ in range(num_iterations):
        result = cv2.matchTemplate(gray_img, gray_template, method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    
    elapsed = (time.time() - start) / num_iterations * 1000  # Convert to ms
    
    timing_results.append({
        'method': method_name,
        'time_ms': elapsed
    })
    
    print(f"{method_name:<25} : {elapsed:>8.3f} ms/iter")

print("="*70)

# Visualize timing results
methods_list = [r['method'] for r in timing_results]
times_list = [r['time_ms'] for r in timing_results]

fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#e74c3c' if 'NORMED' in m else '#3498db' for m in methods_list]
bars = ax.bar(methods_list, times_list, color=colors, alpha=0.7, edgecolor='black')

# Add value labels
for bar, time_val in zip(bars, times_list):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
           f'{time_val:.2f} ms',
           ha='center', va='bottom', fontsize=10, fontweight='bold')

ax.set_ylabel('執行時間 (milliseconds)', fontsize=12, fontweight='bold')
ax.set_title('模板匹配方法性能比較', fontsize=14, fontweight='bold')
ax.set_xticklabels(methods_list, rotation=45, ha='right')
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#3498db', edgecolor='black', label='非歸一化'),
    Patch(facecolor='#e74c3c', edgecolor='black', label='歸一化')
]
ax.legend(handles=legend_elements, loc='upper right', fontsize=11)

plt.tight_layout()
plt.show()

print("\n💡 性能觀察:")
print("- 歸一化方法稍慢，但結果更可靠")
print("- TM_SQDIFF 系列通常最快")
print("- 實際應用中，性能差異通常可以忽略")

<div style="page-break-after: always"></div>

# 9. 限制與解決方案

## 9-1: 模板匹配的主要限制

### 限制 1: 旋轉不變性 ❌

> **問題**: 模板匹配無法處理旋轉變化
>
> **解決方案**:
> 1. **多角度匹配**: 旋轉模板並嘗試多個角度
> 2. **特徵匹配**: 使用 SIFT/SURF/ORB 等旋轉不變特徵
> 3. **深度學習**: 使用 CNN 進行物體檢測

### 限制 2: 尺度不變性 ❌

> **問題**: 目標大小變化時匹配失敗
>
> **解決方案**:
> 1. **多尺度匹配**: 如本模組第 6 節所示
> 2. **圖像金字塔**: 構建多尺度圖像金字塔
> 3. **特徵匹配**: SIFT/SURF 具有尺度不變性

### 限制 3: 光照變化敏感 ⚠️

> **問題**: 光照條件變化影響匹配
>
> **解決方案**:
> 1. **使用歸一化方法**: TM_CCOEFF_NORMED 最魯棒
> 2. **直方圖均衡化**: 預處理圖像
> 3. **梯度特徵**: 使用邊緣或梯度進行匹配

### 限制 4: 部分遮擋 ❌

> **問題**: 目標被部分遮擋時匹配失敗
>
> **解決方案**:
> 1. **特徵點匹配**: SIFT/SURF/ORB + RANSAC
> 2. **深度學習**: 使用物體檢測模型
> 3. **多模板匹配**: 使用目標的不同部分作為模板

### 限制 5: 計算效率 ⚠️

> **問題**: 大圖像或多尺度匹配計算量大
>
> **解決方案**:
> 1. **ROI 限制**: 只在感興趣區域搜索
> 2. **圖像降採樣**: 先在低解析度匹配，再精確定位
> 3. **GPU 加速**: 使用 CUDA 加速（cv2.cuda 模組）
> 4. **快速匹配**: 使用更快的特徵檢測器

## 9-2: 方法選擇決策樹

```
目標特性分析
│
├─ 固定大小、固定方向、簡單背景?
│   └─ 是 → 使用模板匹配 (TM_CCOEFF_NORMED)
│
├─ 有尺度變化?
│   └─ 是 → 多尺度模板匹配 或 特徵匹配
│
├─ 有旋轉變化?
│   └─ 是 → 特徵匹配 (SIFT/SURF/ORB) 或 深度學習
│
├─ 有遮擋?
│   └─ 是 → 特徵匹配 + RANSAC 或 深度學習
│
└─ 複雜場景、多類別?
    └─ 是 → 深度學習 (YOLO/SSD/Faster R-CNN)
```

<div style="page-break-after: always"></div>

# 10. 總結與延伸

## 10-1: 核心要點回顧

### 模板匹配基礎

> **工作原理**:
> * 滑動窗口機制，計算每個位置的相似度
> * 結果矩陣尺寸: (W-w+1) × (H-h+1)
> * 簡單易用，OpenCV 內建支援

### 六種匹配方法

> **推薦順序**:
> 1. **TM_CCOEFF_NORMED** ⭐⭐⭐⭐⭐ - 最佳選擇
> 2. **TM_CCOEFF** ⭐⭐⭐ - 不需要歸一化時使用
> 3. **TM_SQDIFF_NORMED** ⭐⭐ - 備選方案
> 4. 其他方法 - 特殊情況使用

### 進階技術

> **多目標檢測**:
> * 閾值過濾 + 非極大值抑制 (NMS)
> * 適合檢測多個相同物體
>
> **多尺度匹配**:
> * 解決尺度變化問題
> * 在多個尺度上搜索最佳匹配
>
> **光照魯棒性**:
> * TM_CCOEFF_NORMED 最魯棒
> * 考慮使用直方圖均衡化預處理

## 10-2: 實戰應用指南

### 適用場景

> | 應用 | 難度 | 推薦方法 | 備註 |
> |------|------|---------|------|
> | Logo 檢測 | ⭐ | TM_CCOEFF_NORMED | 簡單直接 |
> | UI 自動化 | ⭐ | TM_CCOEFF_NORMED + NMS | 適合固定 UI |
> | 工業檢測 | ⭐⭐ | 多尺度 + NMS | 需要精確定位 |
> | 遊戲機器人 | ⭐⭐ | TM_CCOEFF_NORMED | 實時性要求 |
> | 監控系統 | ⭐⭐⭐ | 特徵匹配 | 環境變化大 |

### 參數調優建議

> **閾值設定**:
> * TM_CCOEFF_NORMED: 0.7 ~ 0.9 (越高越嚴格)
> * 調整策略: 從高閾值開始，逐步降低直到滿足需求
>
> **NMS 參數**:
> * IoU 閾值: 0.3 ~ 0.5 (越小去除越多)
> * 根據目標密集程度調整
>
> **多尺度範圍**:
> * 尺度範圍: 0.5 ~ 2.0 (根據實際情況)
> * 尺度步長: 0.1 (平衡精度與速度)

## 10-3: 性能優化技巧

> **1. 圖像預處理**:
> ```python
> # 轉換為灰階
> gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
> 
> # 降採樣（速度優先）
> small = cv2.resize(gray, None, fx=0.5, fy=0.5)
> 
> # 直方圖均衡化（魯棒性優先）
> equalized = cv2.equalizeHist(gray)
> ```
>
> **2. ROI 限制**:
> ```python
> # 只在感興趣區域搜索
> roi = image[y1:y2, x1:x2]
> result = cv2.matchTemplate(roi, template, method)
> ```
>
> **3. 粗到細策略**:
> ```python
> # 第一步：低解析度快速定位
> small_img = cv2.resize(image, None, fx=0.25, fy=0.25)
> small_template = cv2.resize(template, None, fx=0.25, fy=0.25)
> result = cv2.matchTemplate(small_img, small_template, method)
> 
> # 第二步：在原圖上精確定位
> rough_location = cv2.minMaxLoc(result)[3]
> refined_roi = image[rough_y:rough_y+h, rough_x:rough_x+w]
> final_result = cv2.matchTemplate(refined_roi, template, method)
> ```

## 10-4: 何時不使用模板匹配

> **考慮其他方法的情況**:
>
> 1. **目標有旋轉變化** → 使用特徵匹配 (SIFT/SURF/ORB)
> 2. **複雜背景** → 使用深度學習物體檢測
> 3. **多類別物體** → 使用分類或檢測模型
> 4. **需要語義理解** → 使用深度學習
> 5. **實時性要求極高** → 考慮硬體加速或更快算法

## 10-5: 延伸學習

### 後續模組預告

> **4.2.2 特徵匹配 (Feature Matching)**:
> * SIFT/SURF/ORB 特徵檢測與描述
> * Brute-Force 與 FLANN 匹配
> * RANSAC 外點過濾
> * 單應性矩陣 (Homography) 估計
>
> **4.3 物體追蹤 (Object Tracking)**:
> * 光流法追蹤
> * MeanShift / CamShift 追蹤
> * KCF / CSRT 追蹤器
> * 多目標追蹤
>
> **5.1 深度學習物體檢測**:
> * YOLO / SSD / Faster R-CNN
> * 預訓練模型使用
> * 自定義物體檢測

### 推薦資源

> **官方文檔**:
> * OpenCV Template Matching: https://docs.opencv.org/4.x/d4/dc6/tutorial_py_template_matching.html
> * OpenCV Feature Detection: https://docs.opencv.org/4.x/db/d27/tutorial_py_table_of_contents_feature2d.html
>
> **論文**:
> * "Template Matching Techniques: A Survey" - Essam A. Rashed
> * "Distinctive Image Features from Scale-Invariant Keypoints" - David Lowe (SIFT)
>
> **實用工具**:
> * OpenCV Documentation
> * LearnOpenCV Blog
> * PyImageSearch Tutorials

## 10-6: 實作練習建議

### 初級練習

> 1. **基礎匹配**: 實作簡單的模板匹配，比較不同方法
> 2. **閾值調整**: 嘗試不同閾值，觀察結果變化
> 3. **多目標檢測**: 實作 NMS 算法

### 中級練習

> 4. **多尺度匹配**: 實作完整的多尺度模板匹配
> 5. **UI 自動化**: 開發簡單的 UI 元素定位工具
> 6. **性能優化**: 實作粗到細策略，比較性能提升

### 高級練習

> 7. **旋轉匹配**: 實作多角度模板匹配
> 8. **混合方法**: 結合模板匹配與特徵匹配
> 9. **實際項目**: 開發完整的物體檢測應用

---

## 🎯 總結

模板匹配是計算機視覺中最基礎且實用的技術之一。雖然有其限制，但在適合的場景下，它仍然是最簡單、最有效的解決方案。

**關鍵建議**:
- ✅ 優先使用 TM_CCOEFF_NORMED
- ✅ 多目標檢測必須使用 NMS
- ✅ 尺度變化使用多尺度匹配
- ✅ 了解限制，選擇合適方法
- ✅ 性能優化從圖像預處理開始

**下一步**: 繼續學習 4.2.2 特徵匹配模組，掌握更強大、更魯棒的物體檢測技術。

---

**模組完成！繼續學習 4.2.2 特徵匹配，探索旋轉不變和尺度不變的特徵檢測方法。**